# T1547.014 - Active Setup
Adversaries may achieve persistence by adding a Registry key to the Active Setup of the local machine. Active Setup is a Windows mechanism that is used to execute programs when a user logs in. The value stored in the Registry key will be executed after a user logs into the computer.(Citation: Klein Active Setup 2010) These programs will be executed under the context of the user and will have the account's associated permissions level.

Adversaries may abuse Active Setup by creating a key under <code> HKLM\SOFTWARE\Microsoft\Active Setup\Installed Components\</code> and setting a malicious value for <code>StubPath</code>. This value will serve as the program that will be executed when a user logs into the computer.(Citation: Mandiant Glyer APT 2010)(Citation: Citizenlab Packrat 2015)(Citation: FireEye CFR Watering Hole 2012)(Citation: SECURELIST Bright Star 2015)(Citation: paloalto Tropic Trooper 2016)

Adversaries can abuse these components to execute malware, such as remote access tools, to maintain persistence through system reboots. Adversaries may also use [Masquerading](https://attack.mitre.org/techniques/T1036) to make the Registry entries look as if they are associated with legitimate programs.

## Atomic Tests

### Atomic Test #1 - HKLM - Add atomic_test key to launch executable as part of user setupThis test will create an "atomic_test" key under 'HKLM:\SOFTWARE\Microsoft\Active Setup\Installed Components' to launch calc by configuring an active setup executable and 
forcing to run active setup using the "runonce.exe /AlternateShellStartup" command. 
Without the "runonce.exe /AlternateShellStartup" command it would run during the next logon for each user.

Note: If you logout before running the cleanup command, you will be required to go through the OOBE (out-of-box experience) setup sequence to log back in. 
The payload will only run once unless the cleanup command is run in between tests.

[Active Setup Explained](https://helgeklein.com/blog/active-setup-explained/)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-Item "HKLM:\SOFTWARE\Microsoft\Active Setup\Installed Components" -Name "atomic_test" -Force
Set-ItemProperty "HKLM:\SOFTWARE\Microsoft\Active Setup\Installed Components\atomic_test" "(Default)" "ART TEST" -Force
Set-ItemProperty "HKLM:\SOFTWARE\Microsoft\Active Setup\Installed Components\atomic_test" "StubPath" "C:\Windows\System32\calc.exe" -Force 
& $env:SYSTEMROOT\system32\runonce.exe /AlternateShellStartup```

In [ ]:
Invoke-AtomicTest T1547.014 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item "HKLM:\SOFTWARE\Microsoft\Active Setup\Installed Components\atomic_test" -Force -ErrorAction Ignore
Remove-Item "HKCU:\SOFTWARE\Microsoft\Active Setup\Installed Components\atomic_test" -Force -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1547.014 -TestNumbers 1 -Cleanup

### Atomic Test #2 - HKLM - Add malicious StubPath value to existing Active Setup EntryThis test will add a StubPath entry to the Active Setup native registry key associated with 'Internet Explorer Core Fonts' (UUID {C9E9A340-D1F1-11D0-821E-444553540600}) 
Said key doesn't have a StubPath value by default, by adding one it will launch calc by forcing to run active setup using runonce.exe /AlternateShellStartup. 
Without the last command it will normally run on next user logon. Note: this test will only run once successfully if no cleanup command is run in between test.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ItemProperty "HKLM:\SOFTWARE\Microsoft\Active Setup\Installed Components\{C9E9A340-D1F1-11D0-821E-444553540600}" "StubPath" "C:\Windows\System32\calc.exe" -Force
& $env:SYSTEMROOT\system32\runonce.exe /AlternateShellStartup```

In [ ]:
Invoke-AtomicTest T1547.014 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-ItemProperty "HKLM:\SOFTWARE\Microsoft\Active Setup\Installed Components\{C9E9A340-D1F1-11D0-821E-444553540600}" -Name "StubPath" -Force
Remove-ItemProperty "HKCU:\SOFTWARE\Microsoft\Active Setup\Installed Components\{C9E9A340-D1F1-11D0-821E-444553540600}" -Name "Version" -Force```

In [ ]:
Invoke-AtomicTest T1547.014 -TestNumbers 2 -Cleanup

### Atomic Test #3 - HKLM - re-execute 'Internet Explorer Core Fonts' StubPath payload by decreasing version numberThis test will decrease the version number of the 'Internet Explorer Core Fonts' (UUID {C9E9A340-D1F1-11D0-821E-444553540600}) registry key for the current user, 
which will force the StubPath payload (if set) to execute.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ItemProperty -Path "HKCU:\SOFTWARE\Microsoft\Active Setup\Installed Components\{C9E9A340-D1F1-11D0-821E-444553540600}" -Name "Version" -Value "0,0,0,0"
& $env:SYSTEMROOT\system32\runonce.exe /AlternateShellStartup
```

In [ ]:
Invoke-AtomicTest T1547.014 -TestNumbers 3

## Detection
Monitor Registry key additions and/or modifications to <code>HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Active Setup\Installed Components\</code>.

Tools such as Sysinternals Autoruns may also be used to detect system changes that could be attempts at persistence, including listing the Active Setup Registry locations and startup folders.(Citation: TechNet Autoruns) Suspicious program execution as startup programs may show up as outlier processes that have not been seen before when compared against historical data.